In [1]:
# coding=utf-8

import os
import time
import random
import logging
import subprocess
from multiprocessing import Process, Pool, Queue

"""
import os
import subprocess
# cmd 清屏
i=os.system("cls")
i=subprocess.call("cls", shell=True)
"""

'''
a = 5
print('Process (%s) start...' % os.getpid())
# Only works on Unix/Linux/Mac:
pid = os.fork()   # fork() 主进程中调用一次，主进程中返回一次，子进程中返回一次！
a = a -1          # 常见的Apache服务器就是由父进程监听端口，每当有新的http请求时，就fork出子进程来处理新的http请求。
if pid == 0:      
    # fork() 做了进程克隆，并在该方法内分裂, 分裂之后会导致返回两次，以及os.fork()之后的代码被执行两次（Ubuntu中已被验证）
    print('I am child process (%s) and my parent is %s.' % (os.getpid(), os.getppid()))
else:             # 进程fork不会共享fork之前的资源，是对fork之前的所有资源的拷贝
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid), a)
'''

# print(__name__, '正在被导入执行！')
#   跨平台的多进程


class MultiProcessTest(object):
    # print('-------%s---' %(multiprocessing.cpu_count()))
    # 【 猜测  导入的时候除了方法体内代码不执行，其余代码块会全被执行。。。】

    @staticmethod
    def run(name):
        print('当前进程', name, ':', os.getpid(), 'my father is', os.getppid())

    def fork_process(self):
        print('Father will born a subprocess!')
        p = Process(target=self.run, args=('test',))
        # 克隆主进程（import启动它的模块以及复制它的资源），然后开启线程执行target
        p.start()
        p.join()
        # 同java线程一致 ：子线程执行完成之后，主线程才会继续执行
        print('Child process end.')

    @staticmethod
    def long_time_task(name):
        print('Run task %s (%s)...' % (name, os.getpid()))
        start = time.time()
        time.sleep(random.random() * 3)
        end = time.time()
        print('Task %s runs %0.2f seconds.' % (name, (end - start)))

    def fork_multi_process(self):
        print('Parent process %s.' % os.getpid())
        p = Pool()  # 可传入cpu 核数，默认为机器cpu核数
        for i in range(5):
            # Asynchronous version of `apply()` method. 【异步 ？】
            # Equivalent of `func(*args, **kwds)`.      【同步 ？】
            p.apply_async(self.long_time_task, args=(i,))
        print('Waiting for all subprocesses done...')
        p.close()
        p.join()
        #  对Pool对象调用join()方法会等待所有子进程执行完毕，之后才会执行主进程
        #  调用join()之前必须先调用close()，调用close()之后就不能继续添加新的Process了
        print('All subprocesses done.')

    a = 5

    def test_pool_map(self, x):
        # print(self.a, __class__.a, testMultiProcess.a, testMultiProcess, __class__,
        #               self.__class__, testMultiProcess.__class__)
        # print(a)  not defined, 【为什么 __class__ 不会报错？】
        # __class__：【为什么不需要引用前缀？】
        # __all__:
        #  建议直接使用类来引用类成员
        #  类成员变量可以直接通过类名调用！类成员变量是实例共享的！相当于java中的静态变量
        #  在python中引用其它成员一定要指定作用域前缀，没有作用域前缀调用的成员就是当前作用域的成员！
        # python是按顺序解释执行的，不像java可以不用注意成员定义的顺序！
        self.a = self.a - 1
        print(os.getpid(), '----', x, self.a)
        # print 参数不能是一个赋值表达式，在python中赋值意味着变量的重新指向
        # 而在java中变量是固定的，赋值表达式的值就是 = 左边的值
        return x ** x

    def test_pool_map(self):
        with Pool(3) as p:
            # Apply `func` to each element in `iterable`, collecting the results
            # in a list that is returned.
            print(p.map(self.test_pool_map, [1, 2, 3, 4]))


'''
     An attempt has been made to start a new process before the
        current process has finished its bootstrapping phase.

        This probably means that you are not using fork to start your
        child processes and you have forgotten to use the proper idiom
        in the main module:

            if __name__ == '__main__':
                freeze_support()
                ...

        The "freeze_support()" line can be omitted if the program
        is not going to be frozen to produce an executable.

         Windows 上多进程的实现问题。在 Windows 上，子进程会自动  import 启动它的这个文件，
                    而在 import 的时候是会执行这些语句的。 如果你这么写的话就会无限递归子进程报错。所以必须
                    把创建子进程的部分用那个 if 判断保护起来，import 的时候 __name__ 不是 __main__ ，
                    而是 __mp_main__，这样就不会递归运行了。

                   在unix下
            fork, 不会再子进程中导入开启它的文件！继承所有fork之前的资源并与父进程同步执行之后的所有逻辑
            multiprocessing, 不会再子进程中导入开启它的文件,  克隆父进程启动它之前所有资源, 但是只会执行   target

                    【进程里的主线程的理解。。。。】

'''


class TestSubprocess(object):
    #  很多时候，子进程并不是自身，而是一个外部进程。创建了子进程后，还需要控制子进程的输入和输出。
    #  Nslookup是由微软发布用于对DNS服务器进行检测和排错的命令行工具。
    def test_subprocess(self):
        print('$ nslookup www.python.org')
        r = subprocess.call(['nslookup', 'www.python.org'])
        print('Exit code:', r)

        print('$ nslookup')
        p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
        print(output.decode('utf-8'))
        print('Exit code:', p.returncode)
        '''
            set q=mx
            python.org
            exit
        '''

        if __name__ == '__main__':
            try:
                cmd='adb shell getevent -l'
                pipe = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

                for line in pipe.stdout:
                    print(line)
            except Exception as e:
                logging.exception(e)
                print('exception happened', e)
            print('finished')

class ProcessInteractive(object):
    # multiprocessing模块包装了底层的机制，提供了Queue、Pipes等多种方式来交换数据

    @staticmethod
    def write(q):
        print('Process to write: %s' % os.getpid())
        for value in ['A', 'B', 'C']:
            q.put(value)
            time.sleep(random.random())

    @staticmethod
    def read(q):
        print('Process to read: %s' % os.getpid())
        while True:
            value = q.get(True)  # 阻塞的  无法get到就阻塞在这里
            print('Get %s from queue.' % value)

    def interactive(self):
        # 父进程创建Queue，并传给各个子进程：
        q = Queue()
        pw = Process(target=self.write, args=(q,))
        pr = Process(target=self.read, args=(q,))
        # 启动子进程pw，写入:
        pw.start()
        # 启动子进程pr，读取:
        pr.start()
        # 等待pw结束:
        pw.join()
        # pr进程里是死循环，无法等待其结束，只能强行终止:
        pr.terminate()

    '''
        在Unix/Linux下，multiprocessing模块封装了fork()调用，使我们不需要关注fork()的细节。
        由于Windows没有fork调用，因此，multiprocessing需要“模拟”出fork的效果，父进程所有Python
        对象都必须通过pickle序列化再传到子进程去，所以，如果multiprocessing在Windows下调用失败了，
        要先考虑是不是pickle失败了。
    '''


if __name__ == '__main__':
    # 1：__name__是一个系统级别的变量。加双下划线是因为这是系统定义的名字。普通变量不要使用此方式命名变量。
    # 2：Python有很多模块，而这些模块是可以独立运行的！import的时候是要执行所import的模块的。
    # 3：__name__就是标识模块的名字的一个系统变量。假如当前模块是主模块（python 解释器的入口模块），
    #   那么此模块名字就是__main__，通过if判断这样就可以执行“__main__:”后面的主函数内容；假如此模块是被import的，
    #   则此模块名字为文件名字（不加后面的.py），通过if判断这样就会跳过“__mian__:”后面的内容。
    pass

